In [16]:
!pip install  -q 'ibis-framework[mssql,datafusion,duckdb]'
!pip install  -q altair

**_<u><mark>Get connection</mark></u>_**

In [17]:
import ibis
def get_ibis_connection(workspace ,lakehouse ,engine):
    import ibis
    import sempy.fabric as fabric
    workspaceID = fabric.resolve_workspace_id(workspace)
    df = fabric.list_items(workspace=workspace)
    df =df.loc[(df['Display Name'] == lakehouse) & (df['Type'] == 'Lakehouse')]
    lakehouseID = df['Id'].iloc[0]
    if engine =='dwh':
        import struct , pyodbc , ibis.backends.mssql
        dwh_backend = ibis.backends.mssql.Backend()
        client = fabric.FabricRestClient()
        response = client.get(f"/v1/workspaces/{workspaceID}/lakehouses/{lakehouseID}")
        responseJson = response.json()
        sql_endpoint = responseJson['properties']['sqlEndpointProperties']['connectionString']
        token = notebookutils.credentials.getToken('https://analysis.windows.net/powerbi/api').encode("UTF-16-LE")
        token_struct = struct.pack(f'<I{len(token)}s', len(token), token)
        dwh_backend.con = pyodbc.connect(f"Driver={{ODBC Driver 18 for SQL Server}};Server={sql_endpoint},1433;Encrypt=Yes;TrustServerCertificate=No", attrs_before={1256: token_struct})
        dwh_backend.raw_sql(" use NY;")
        return dwh_backend
    else:
        access_token     = notebookutils.credentials.getToken('storage')
        storage_options  = {"bearer_token": access_token, "use_fabric_endpoint": "true"} 
        con = eval(f"ibis.{engine}.connect()")
        base_path = f'abfss://{workspace}@onelake.dfs.fabric.microsoft.com/{lakehouse}.Lakehouse/Tables'
        list_db = [item.name for item in  notebookutils.fs.ls(base_path)]
        for db in list_db:
           con.create_database(db,force =True)
           for table in [item.name for item in  notebookutils.fs.ls(base_path +f'/{db}')]:
             t= con.read_delta(base_path+f'/{db}/{table}',table,storage_options = storage_options)
             con.create_view(table, t, database=f'{db}')      
        return con

In [18]:
#switch between duckdb , datafusion , and dwh
con = get_ibis_connection(workspace ='pythononly',lakehouse ='NY',engine ='datafusion')

In [19]:
con.list_catalogs()

['datafusion']

In [20]:
con.list_databases()

['dbo', 'ny', 'public']

In [21]:
con.list_tables(database = 'ny')

['taxi']

In [22]:
taxi = con.table(name="taxi", database='ny')
agg = taxi.group_by("date").aggregate(TotalFares=taxi.fare_amount.sum(), AVGFares=taxi.fare_amount.mean())
ibis.to_sql(agg)

```sql
SELECT
  "t0"."date",
  SUM("t0"."fare_amount") AS "TotalFares",
  AVG("t0"."fare_amount") AS "AVGFares"
FROM (
  SELECT
    "t0"."trip_distance",
    "t0"."store_and_fwd_flag",
    "t0"."fare_amount",
    "t0"."extra",
    "t0"."mta_tax",
    "t0"."tip_amount",
    "t0"."tolls_amount",
    "t0"."improvement_surcharge",
    "t0"."total_amount",
    "t0"."airport_fee",
    "t0"."congestion_surcharge",
    "t0"."vendorid",
    "t0"."passenger_count",
    "t0"."pulocationid",
    "t0"."dolocationid",
    "t0"."payment_type",
    "t0"."ratecodeid",
    "t0"."tpep_pickup_datetime",
    "t0"."tpep_dropoff_datetime",
    "t0"."hour",
    "t0"."file",
    "t0"."year",
    "t0"."date"
  FROM "ny"."taxi" AS "t0"
) AS t0
GROUP BY
  "t0"."date"
```

In [23]:
data = agg.to_pyarrow()

In [24]:
import altair as alt
brush = alt.selection_interval()
details = alt.Chart(data).mark_bar().encode(alt.X('date:T'),  alt.Y('TotalFares:Q'), tooltip=[alt.Tooltip('date:T',format='%Y-%m-%d %H'),'TotalFares:Q']
).properties( width=1400,  height=400 ).add_params( brush)

summary = alt.Chart(data).mark_square().encode( alt.X('date:T'), alt.Y('AVGFares:Q'), tooltip=['AVGFares:Q'] ).properties(  width=1400,  height=400).transform_filter( brush)
details & summary

alt.VConcatChart(...)